#### A conversational agent that enquires user details and curates a json file from the information

Required format to be curated:\
{ \
  name: "",\
  age: 0,\
  goal: "",\
  educational background: "",\
  level/approach: "",\
  depth of the explanations: "",\
}

In [ ]:
details = {
    "name": "",
    "age": 0,
    "goal": "",
    "educational_background": "",
    "approach": "",
    "topic_depth": "",
}

##### Example conversation chain from langchain docs

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ChatMessageHistory

llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

In [9]:
conversation_with_summary.predict(input="I'm doing great, I was just wondering if you could help me with my English.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks the AI how it is doing and the AI responds that it is helping a customer with a technical issue.
Human: I'm doing great, I was just wondering if you could help me with my English.
AI:

> Finished chain.


" Sure, I'd be happy to help you with your English. What kind of help do you need?"

In [10]:
conversation_with_summary.predict(input="Just explain to me what's the point of english? like in the technical issue you're solving, how is english helping you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks the AI how it is doing and the AI responds that it is helping a customer with a technical issue. The human then asks for help with their English, to which the AI responds positively and inquires what kind of help is needed.
Human: Just explain to me what's the point of english? like in the technical issue you're solving, how is english helping you?
AI:

> Finished chain.


' English is helping me to communicate with the customer and understand their technical issue. It is also helping me to explain the technical issue to the customer in a way that they can understand.'

The Converstation chain class works well but we need a custom chain for this use case

#### Custom conversational chain for details enquiry 

In [51]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist"), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injected
])
    
memory = ConversationSummaryMemory(llm = OpenAI(), memory_key="chat_history", return_messages=True)


llm = ChatOpenAI()

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [52]:
chat_llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
Human: Hi there my friend

> Finished chain.


'Hello! How can I assist you today?'

In [53]:
chat_llm_chain.predict(human_input="Let's get started")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance.
Human: Let's get started

> Finished chain.


"Great! I'm here to assist you. Before we begin, may I ask for your name?"

In [54]:
chat_llm_chain.predict(human_input="I'm Bharath Raj")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and the AI requests the human's name.
Human: I'm Bharath Raj

> Finished chain.


'Hello Bharath Raj! How can I assist you today?'

In [55]:
chat_llm_chain.predict(human_input="I'm 20 years old")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and the AI requests the human's name, which the human provides as Bharath Raj. The AI then greets Bharath Raj and offers its assistance.
Human: I'm 20 years old

> Finished chain.


'Great, Bharath Raj! Could you please share your educational background with me?'

In [56]:
chat_llm_chain.predict(human_input="I'm an undergrad student")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and the AI requests the human's name, which the human provides as Bharath Raj. The AI then greets Bharath Raj and offers its assistance, and the human reveals they are 20 years old. The AI then requests the human's educational background.
Human: I'm an undergrad student

> Finished chain.


"That's great, Bharath Raj! As an undergraduate student, what is your field of study or major?"

In [57]:
chat_llm_chain.predict(human_input="I'm Computer Science")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and the AI requests the human's name, which the human provides as Bharath Raj. The AI then greets Bharath Raj and offers its assistance, and the human reveals they are 20 years old. The AI then requests the human's educational background, to which the human replies they are an undergraduate student. The AI responds positively to this, and then requests to know the human's field of study or major.
Human: I'm Computer Science

> Finished chain.


'Great, Bharath Raj! As a computer science student, what are your goals or aspirations in this field?'

In [58]:
chat_llm_chain.predict(human_input="I want to be a machine learning engineer")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and the AI requests the human's name, which the human provides as Bharath Raj. The AI then greets Bharath Raj and offers its assistance, and the human reveals they are 20 years old. The AI then requests the human's educational background, to which the human replies they are an undergraduate student. The AI responds positively to this, and then requests to know the human's field of study or major, to which the human replies Computer Science. The AI then expresses enthusiasm and asks the human what their goals or aspirations are in Computer Science.
Human: I want to be a machine learning engineer

> F

"That's great to hear, Bharath Raj! Becoming a machine learning engineer is an exciting and promising career choice in the field of Computer Science. To help you achieve your goal, I'd like to know more about your educational background and any previous experience you may have in this field. Could you please share with me your educational background and any relevant courses or projects you have completed so far?"

In [59]:
chat_llm_chain.predict(human_input="Well, I'm good at python, I'm familiar with ML frameworks but I want to get better at Deep learning")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 

The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and the AI requests the human's name, which the human provides as Bharath Raj. The AI then greets Bharath Raj and offers its assistance, and the human reveals they are 20 years old. The AI then requests the human's educational background, to which the human replies they are an undergraduate student. The AI responds positively to this, and then requests to know the human's field of study or major, to which the human replies Computer Science. The AI then expresses enthusiasm and asks the human what their goals or aspirations are in Computer Science, to which the human replies they want to be a machin

"That's great to hear, Bharath Raj! Python is a widely used language in the field of machine learning and having familiarity with ML frameworks is a valuable skill. Deep learning is a fascinating area within machine learning, and it's wonderful that you want to enhance your skills in that domain.\n\nTo help you with deep learning, could you please share some more details about your educational background? Specifically, have you taken any courses related to deep learning or completed any projects in this field?"

In [60]:
chat_llm_chain.predict(human_input="I have not taken any deep learning courses or have done any projects")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and the AI requests the human's name, which the human provides as Bharath Raj. The AI then greets Bharath Raj and offers its assistance, and the human reveals they are 20 years old. The AI then requests the human's educational background, to which the human replies they are an undergraduate student. The AI responds positively to this, and then requests to know the human's field of study or major, to which the human replies Computer Science. The AI then expresses enthusiasm and asks the human what their goals or aspirations are in Computer Science, to which the human replies they want to be a machine

"That's alright, Bharath Raj! Deep learning is a fascinating field, and there's always room to learn and grow. Have you taken any courses or worked on any projects related to machine learning or Python programming in general?"

In [61]:
chat_llm_chain.predict(human_input="Yes, I have taken a beginner's course on ML")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and provides their name as Bharath Raj and reveals they are 20 years old and an undergraduate student studying Computer Science. The AI expresses enthusiasm and asks the human what their goals or aspirations are in Computer Science, to which the human replies they want to be a machine learning engineer. The AI requests more information about the human's educational background and any relevant courses or projects they have completed so far. The human replies that they are good at Python, familiar with ML frameworks, and want to get better at Deep learning. The AI encourages the human to share more de

"That's great, Bharath Raj! Taking a beginner's course on machine learning is a good starting point. Can you tell me more about the course you took? What were some of the topics covered and any projects or assignments you worked on?"

In [62]:
chat_llm_chain.predict(human_input="Just linear regression, logistic regression, decision trees, random forests, SVMs, KNNs, K-means clustering, PCA, and some other stuff")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are the Topic Guide bot interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions. You ask the questions, you do not assist
System: 
The human greets the AI and the AI responds with an offer for assistance. The human agrees to get started and provides their name as Bharath Raj and reveals they are 20 years old and an undergraduate student studying Computer Science. The AI expresses enthusiasm and asks the human what their goals or aspirations are in Computer Science, to which the human replies they want to be a machine learning engineer. The AI requests more information about the human's educational background and any relevant courses or projects they have completed so far. The human replies that they are good at Python, familiar with ML frameworks, and want to get better at Deep learning. The AI encourages the human to share more de

"That sounds like a solid foundation in machine learning! It's great that you have covered a variety of algorithms and techniques. Can you provide more details about the beginner's course you took? What topics did it cover and did you work on any projects or assignments related to deep learning?"